# QSANNdRA tutorial

Creator: Dominika Ďurovčíková

This notebook provides a tutorial on how to set up and run the QSANNdRA model for predicting quasar continua. This code was developed and first implemented in [Ďurovčíková et al. 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.493.4256D/abstract), and it was most recently updated as a result of [Ďurovčíková et al. 2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240110328D/abstract).

Please cite both references if you use this code.

## Setup

Set your parameters:

In [ ]:
version = '0' # for versioning purposes

lam_in = [1290, 2400] # input wavelength range in AA
lam_out = [1050, 1290] # output/prediction wavelength range in AA

lam_norm = 1290 # wavelength at which to normalize spectra to 1

snr_thres = 3.0 

Import necessary packages:

In [ ]:
import os
import numpy as np 
from astropy.io import fits 
import pandas as pd
import preprocessing as pr
import QSANNdRA as Q
import QSmooth as QS

Create the directory structure:

In [ ]:
if os.path.isdir('data/') == False:
    os.mkdir('data/')
if os.path.isdir('data/dataframes/') == False:
    os.mkdir('data/dataframes/')
if os.path.isdir('data/dataframes/'+str(version)) == False:
    os.mkdir('data/dataframes/'+str(version))
if os.path.isdir('models/') == False:
    os.mkdir('models/')
if os.path.isdir('models/'+str(version)+'/') == False:
    os.mkdir('models/'+str(version)+'/')
if os.path.isdir('models/'+str(version)+'/errors/') == False:
    os.mkdir('models/'+str(version)+'/errors/')
if os.path.isdir('models/'+str(version)+'/preprocessing_models/') == False:
    os.mkdir('models/'+str(version)+'/preprocessing_models/')
if os.path.isdir('models/'+str(version)+'/QSANNdRA/') == False:
    os.mkdir('models/'+str(version)+'/QSANNdRA/')
if os.path.isdir('plots/') == False:
    os.mkdir('plots/')
if os.path.isdir('plots/preprocessing/') == False:
    os.mkdir('plots/preprocessing/')
if os.path.isdir('plots/preprocessing/'+version+'/') == False:
    os.mkdir('plots/preprocessing/'+version+'/')

## Downloading catalog and SDSS training data

In [ ]:
catalog = 'catalogs/DR16Q_v4.fits'
fits_directory = 'SDSS_quasars/'
tag = '_'+version

if os.path.isdir('SDSS_quasars/') == False:
    os.mkdir('SDSS_quasars/')

# relevant redshifts to cover desired wavelength range in SDSS
z_start = 3600./lam_out[0] - 1
z_end = 10000./lam_in[1] - 1 
print('fetching SDSS spectra in the redshift range '+str(z_start)+'-'+str(z_end))

qsos = pr.perform_catalog_cut(catalog=catalog, z_start=z_start, z_end=z_end, snr=snr_thres, out_file=None, uncertain_BALs=False)
pr.download_spec_files(qsos,fits_directory,tag)

print('Run the following in the terminal (for Macs):')
print('cat download_SDSS'+tag+'.txt | xargs -n 1 -P 10 wget --directory-prefix='+fits_directory)

## Training set preparation

In [ ]:
# Preprocess training data according to Section 2.1 in Ďurovčíková et al. 2020.
# Note: inspect plots in /plots/preprocessing/+version+/ and adjust smoothing parameters (smooth_b, smooth_r, thr) in pr.preprocess_training_data() accordingly

Df, Df_norm = pr.preprocess_training_data(qsos=qsos, input_directory=fits_directory, output_spectra_file='data/dataframes/'+str(version)+'/training_spectra_all.csv', output_norm_file='data/dataframes/'+str(version)+'/training_spectra_norms_all.csv', SN_threshold=snr_thres, norm_lam=lam_norm, lam_in=lam_in, lam_out=lam_out, version=version)
Df = pd.read_csv('data/dataframes/'+str(version)+'/training_spectra_all.csv')
Df_norm = pd.read_csv('data/dataframes/'+str(version)+'/training_spectra_norms_all.csv')
Df_new = pr.perform_flux_cut(Df=Df, thres_lam=1280)
Df_train, Df_train_norm = pr.prepare_training_set(input_spectra_file=Df_new,input_norm_file=Df_norm,output_spectra_file='data/dataframes/'+str(version)+'/training_spectra.csv',\
    output_norm_file='data/dataframes/'+str(version)+'/training_spectra_norms.csv',lam_in=lam_in,lam_out=lam_out)

Df = []
Df_new = []
Df_norm = []

# Refine the training set with Random Forest predictions as described in Section 2.2 in Ďurovčíková et al. 2020 and save both cleaned and rejected datasets.

spectra_clean, norms_clean, spectra_bad, norms_bad = pr.RF_cleanup(spectra_file='data/dataframes/'+str(version)+'/training_spectra.csv',norms_file='data/dataframes/'+str(version)+'/training_spectra_norms.csv',\
    path_to_clean_spectra='data/dataframes/'+str(version)+'/training_spectra_cleaned.csv',path_to_bad_spectra='data/dataframes/'+str(version)+'/training_spectra_bad.csv',\
        path_to_clean_norms='data/dataframes/'+str(version)+'/training_spectra_norms_cleaned.csv',path_to_bad_norms='data/dataframes/'+str(version)+'/training_spectra_norms_bad.csv')


pr.display_random_spectra('cleaned',lam_out,n=100,version=version)
pr.display_random_spectra('bad',lam_out,n=100,version=version)


## Setting up and training QSANNdRA

In [ ]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

training_spectra = 'data/dataframes/'+str(version)+'/training_spectra_cleaned.csv'
training_norms = 'data/dataframes/'+str(version)+'/training_spectra_norms_cleaned.csv'
path_to_models = 'models/'+str(version)+'/'

if os.path.isdir(path_to_models) == False:
    os.mkdir(path_to_models)

preds, mean_preds = Q.get_QSANNdRA(training_spectra=training_spectra,training_norms=training_norms,models_save=True,path_to_models=path_to_models,C=100,load=False,save_errs=True)

Q.get_QSANNdRA(training_spectra=training_spectra,training_norms=training_norms,models_save=False,path_to_models=path_to_models,C=100,load=True,save_errs=False)

Q.plot_QSANNdRA_performance(path_to_models,training_spectra,training_norms,lam_out)

## Applying QSANNdRA to new quasars

This step will depend on the specifics of the data you want to apply QSANNdRA to. Here, I show how to apply the trained model to a random SDSS spectrum.

In [ ]:
# function that opens the .fits file of the target spectrum, smooths it, and extracts the blue and red sides as well as the normalization at 1290 AA
# adjust to the specifics of your data!!
# in particular smooth_b, smooth_r, and thr should be adjusted for an optimal extraction of the red-side continuum

def prepare_spectrum(input_directory,filename,lam_in,lam_out,norm_lam,smooth_b=0.99999999,smooth_r=0.9999999,thr=2.0):
    norm_loglam = np.around(np.log10(norm_lam),decimals=4)
    loglam_target_in = np.around(np.arange(np.log10(lam_in[0]),np.log10(lam_in[1]),0.0001),decimals=4)
    loglam_target_out = np.around(np.arange(np.log10(lam_out[0]),np.log10(lam_out[1]),0.0001),decimals=4)

    spec = fits.open(str(input_directory)+str(filename))
    z = spec[2].data['Z']
    mask = spec[1].data['and_mask']
    flux = spec[1].data['flux'][mask == 0]
    wave = (10**spec[1].data['loglam'][mask == 0])/(1+z)
    ivar = spec[1].data['ivar'][mask == 0]
    spec.close()

    # smooth
    print(smooth_b)
    int_spec, outlier_mask = QS.spline_smooth(np.log10(wave),flux,1/np.sqrt(ivar),smooth_b,smooth_r,thr)

    # input range
    int_flux = int_spec(loglam_target_in) # interpolate to target input wavelengths
    spec_id = [str(filename)]
    df = pd.DataFrame(data=[int_flux],columns=loglam_target_in,index=spec_id)
    norm_flux = df.iloc[0].at[norm_loglam]
    df_norm = df.divide(norm_flux)
    df_norm = df_norm.drop(columns=norm_loglam)
    red = df_norm.loc[:,:].values[:,:]

    # output range
    int_flux = int_spec(loglam_target_out) # interpolate to target output wavelengths
    spec_id = [str(filename)]
    df = pd.DataFrame(data=[int_flux],columns=loglam_target_out,index=spec_id)
    df_norm = df.divide(norm_flux)
    try:
        df_norm = df_norm.drop(columns=norm_loglam)
    except:
        print('exception, nothing to worry about')
    blue = df_norm.loc[:,:].values[:,:]
    
    return blue, red, norm_flux


In [ ]:
# choose a random spectrum and prepare directories
np.random.seed(1)
tid = np.random.choice(os.listdir(fits_directory))
path_apply = 'data/apply/'
if os.path.isdir(path_apply) == False:
    os.mkdir(path_apply)
if os.path.isdir(path_apply+str(tid)[:-5]) == False:
    os.mkdir(path_apply+str(tid)[:-5])

# preprocess spectrum
blue, red, norm = prepare_spectrum(fits_directory,tid,lam_in=lam_in,lam_out=lam_out,norm_lam=lam_norm,smooth_b=0.99999999,smooth_r=0.9999999,thr=2.0)

tid = tid[:-5]

mean_prediction, predictions = Q.apply_QSANNdRA(path_to_files=path_apply,red=red[:,1:],lam_out=lam_out,norm_lam=lam_norm,path_to_models=path_to_models,specname=tid,norm=norm)

Q.plot_predictions(path_to_files=fits_directory,path_to_plots=path_apply,specname=tid,lam_in=lam_in,lam_out=lam_out,red=red,predictions=predictions,mean_prediction=mean_prediction,norm=norm)